In [ ]:
import os
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


tokenizer.add_special_tokens({'pad_token': '[PAD]'})


data_directory = "/content/varun_V/data"


def clean_and_preprocess(text):

    cleaned_text = text.replace("\n", " ")
    return cleaned_text


text_corpus = []


for file_name in os.listdir(data_directory):
    if file_name.endswith(".txt"):
        file_path = os.path.join(data_directory, file_name)
        with open(file_path, "r", encoding="utf-8") as file:

            file_content = file.read()


            cleaned_content = clean_and_preprocess(file_content)


            text_corpus.append(cleaned_content)


combined_text_corpus = "\n".join(text_corpus)


tokenized_text = tokenizer(combined_text_corpus, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import os
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


tokenizer.add_special_tokens({'pad_token': '[PAD]'})


data_directory = "/content/varun_V/data"


def clean_and_preprocess(text):

    cleaned_text = text.replace("\n", " ")
    return cleaned_text


text_corpus = []


for file_name in os.listdir(data_directory):
    if file_name.endswith(".txt"):
        file_path = os.path.join(data_directory, file_name)
        with open(file_path, "r", encoding="utf-8") as file:

            file_content = file.read()


            cleaned_content = clean_and_preprocess(file_content)


            text_corpus.append(cleaned_content)


combined_text_corpus = "\n".join(text_corpus)


output_file_path = "/content/varun_V/text_corpus.txt"


with open(output_file_path, "w", encoding="utf-8") as output_file:
    output_file.write(combined_text_corpus)

print("Combined text corpus saved successfully to:", output_file_path)

Combined text corpus saved successfully to: /content/varun_V/text_corpus.txt


In [ ]:
import re

def read_qa_pairs(file_path):
    qa_pairs = []
    with open(file_path, 'r') as file:
        data = file.read()
        matches = re.findall(r'Question:(.*?)Answer:(.*?)Question:', data, re.DOTALL)
        for match in matches:
            question = match[0].strip()
            answer = match[1].strip()
            qa_pairs.append((question, answer))
    return qa_pairs

def write_qa_pairs(qa_pairs, output_file):
    with open(output_file, 'w') as file:
        for i, (question, answer) in enumerate(qa_pairs, start=1):
            file.write(f"Question: {question}\n")
            file.write(f"Answer: {answer}\n\n")

file_path = '/content/varun_V/op.txt'
output_file = '/content/varun_V/output.txt'
qa_pairs = read_qa_pairs(file_path)

write_qa_pairs(qa_pairs, output_file)

In [ ]:
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


qna_pairs = []
with open("/content/varun_V/op.txt", "r") as file:
    data = file.read()
    qna_pairs = re.findall(r'Question:\s*(.*?)\s*Answer:\s*(.*?)\s*(?:\n\n|\Z)', data, re.DOTALL)


text_corpus = ""
with open("/content/varun_V/text_corpus.txt", "r") as file:
    text_corpus = file.read()


max_length = 512
tokenized_qna_pairs = []
for question, answer in qna_pairs:
    input_text = f"Question: {question} Answer: {answer}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    tokenized_qna_pairs.append(input_ids)

tokenized_text_corpus = tokenizer.encode(text_corpus, return_tensors="pt", max_length=max_length, truncation=True)


for i in range(len(tokenized_qna_pairs)):
    tokenized_qna_pairs[i] = torch.nn.functional.pad(tokenized_qna_pairs[i], (0, max_length - tokenized_qna_pairs[i].shape[1]), 'constant', 0)


combined_input_ids = torch.cat(tokenized_qna_pairs + [tokenized_text_corpus], dim=0)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
model.train()
for epoch in range(3):
    optimizer.zero_grad()
    outputs = model(input_ids=combined_input_ids, labels=combined_input_ids)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


model.save_pretrained("/content/varun_V/fine_tuned_lm_model")


def generate_answer(question):
    input_text = f"Question: {question}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)


print("Ask questions to the model. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    answer = generate_answer(user_input)
    print("Bot:", answer)

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("/content/varun_V/fine_tuned_lm_model")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.eval()


def generate_text(prompt, max_length=100, temperature=1.0):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=max_length, temperature=temperature)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


prompts = [
    "Question: What are the symptoms of glaucoma?",
    "Question: How can I prevent glaucoma?",
    "Question: What treatments are available for glaucoma?",
    "Question: Is glaucoma hereditary?",
    "Question: Can glaucoma be cured?"
]


for prompt in prompts:
    generated_text = generate_text(prompt, max_length=200, temperature=0.7)
    print("Prompt:", prompt)
    print("Generated Text:")
    print(generated_text)
    print("=" * 50)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

OSError: Incorrect path_or_model_id: '/content/varun_V/fine_tuned_lm_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
print("Ask questions to the model. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    answer = generate_answer(user_input)
    print("Bot:", answer)

Ask questions to the model. Type 'exit' to quit.
You: disease 


NameError: name 'generate_answer' is not defined